In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from tqdm.auto import tqdm


BIG_DATA_FILE = 'drive/MyDrive/DATASETS/PFE/big_data/comptages-clean.parquet'
OUTPUT_DIR = Path('drive/MyDrive/DATASETS/PFE/donnees_entrainement_k_24h_history_v2')

NUM_AMONT = 10
NUM_AVAL = 10

def load_data(file_path):
    print(f"Chargement des données depuis {file_path}...")
    try:
        columns_to_load = ['iu_ac', 't_1h', 'k', 'iu_nd_amont', 'iu_nd_aval']
        df = pd.read_parquet(file_path, columns=columns_to_load)
        df['k'] = pd.to_numeric(df['k'], errors='coerce')
        df.dropna(subset=['k', 'iu_ac', 'iu_nd_amont', 'iu_nd_aval'], inplace=True)
        df['t_1h'] = pd.to_datetime(df['t_1h'])
        df['iu_ac'] = df['iu_ac'].astype(str)
        df['iu_nd_amont'] = df['iu_nd_amont'].astype(str)
        df['iu_nd_aval'] = df['iu_nd_aval'].astype(str)
        df.set_index(['iu_ac', 't_1h'], inplace=True)
        df.sort_index(inplace=True)
        print("Données chargées et indexées.")
        return df
    except FileNotFoundError:
        print(f"ERREUR: Le fichier de données principal '{file_path}' est introuvable.")
        return None
    except Exception as e:
        print(f"Une erreur est survenue lors du chargement des données : {e}")
        return None

def build_graph_from_data(df):
    print("Construction de la topologie du graphe à partir des données...")
    arc_nodes = df.reset_index()[['iu_ac', 'iu_nd_amont', 'iu_nd_aval']].drop_duplicates(subset=['iu_ac'])
    from_amont_node = arc_nodes.groupby('iu_nd_amont')['iu_ac'].apply(list).to_dict()
    to_aval_node = arc_nodes.groupby('iu_nd_aval')['iu_ac'].apply(list).to_dict()

    graph = {}
    for _, row in tqdm(arc_nodes.iterrows(), total=len(arc_nodes), desc="Création du graphe"):
        target_arc = row['iu_ac']
        amont_node = row['iu_nd_amont']
        aval_node = row['iu_nd_aval']
        amont_neighbors = [arc for arc in to_aval_node.get(amont_node, []) if arc != target_arc]
        aval_neighbors = [arc for arc in from_amont_node.get(aval_node, []) if arc != target_arc]
        graph[target_arc] = {'amont': amont_neighbors, 'aval': aval_neighbors}

    print("Graphe construit avec succès.")
    return graph

    # Fonction principale pour générer un fichier d'entraînement par arc,
    # contenant exactement 24 mesures pour une seule journée complète.
def create_daily_training_data():
    df = load_data(BIG_DATA_FILE)
    if df is None: return

    graph = build_graph_from_data(df)
    if not graph: return

    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    print(f"Les fichiers de sortie seront sauvegardés dans : {OUTPUT_DIR}")

    all_arcs = df.index.get_level_values('iu_ac').unique()
    files_created_count = 0

    for target_arc_id in tqdm(all_arcs, desc="Traitement des arcs"):
        try:
            df_arc = df.loc[target_arc_id].copy()
        except KeyError:
            continue

        
        all_days = df_arc.index.normalize().unique()

        # Parcourir les jours en commençant par le plus récent pour trouver un jour valide
        for day in reversed(all_days):
            end_check = day.replace(hour=23, minute=0, second=0)
            start_check = end_check - pd.Timedelta(hours=47)

            expected_range = pd.date_range(start=start_check, end=end_check, freq='h')

            # Vérifier si toutes les 48 heures requises sont présentes dans les données de l'arc
            if all(t in df_arc.index for t in expected_range):
                instances_for_this_arc = []

                for hour in range(24):
                    target_t = day.replace(hour=hour)
                    features_t = target_t - pd.Timedelta(hours=1)
                    history_start_t = features_t - pd.Timedelta(hours=23)

                    history_k_series = df_arc.loc[history_start_t:features_t, 'k']
                    target_k = df_arc.loc[target_t, 'k']

                    if pd.isna(target_k) or history_k_series.isnull().any():
                        instances_for_this_arc = [] # Invalider ce jour si une valeur manque
                        break

                    instance = {
                        'target_iu_ac': target_arc_id,
                        'target_t': target_t,
                        'target_k': target_k,
                        'features_t': features_t,
                    }

                    for i in range(24):
                        instance[f'k_history_{i}'] = history_k_series.iloc[i]

                    neighbors = graph.get(target_arc_id, {})
                    amont_arcs = neighbors.get('amont', [])
                    aval_arcs = neighbors.get('aval', [])

                    for i in range(NUM_AMONT):
                        val = 0.0
                        if i < len(amont_arcs):
                            try:
                                val = df.loc[(amont_arcs[i], features_t), 'k']
                            except KeyError: pass
                        instance[f'amont_k_{i}'] = val if not pd.isna(val) else 0.0

                    for i in range(NUM_AVAL):
                        val = 0.0
                        if i < len(aval_arcs):
                            try:
                                val = df.loc[(aval_arcs[i], features_t), 'k']
                            except KeyError: pass
                        instance[f'aval_k_{i}'] = val if not pd.isna(val) else 0.0

                    instances_for_this_arc.append(instance)

                # Si nous avons bien 24 instances, on sauvegarde et on arrête de chercher un jour pour cet arc
                if len(instances_for_this_arc) == 24:
                    df_output = pd.DataFrame(instances_for_this_arc)
                    output_path = OUTPUT_DIR / f"{target_arc_id}.parquet"
                    df_output.to_parquet(output_path, index=False)
                    files_created_count += 1
                    break  
    print("\nTraitement terminé.")
    print(f"{files_created_count} fichiers ont été créés dans {OUTPUT_DIR}.")

if __name__ == '__main__':
    create_daily_training_data()

Chargement des données depuis drive/MyDrive/DATASETS/PFE/big_data/comptages-clean.parquet...
Données chargées et indexées.
Construction de la topologie du graphe à partir des données...


Création du graphe:   0%|          | 0/1539 [00:00<?, ?it/s]

Graphe construit avec succès.
Les fichiers de sortie seront sauvegardés dans : drive/MyDrive/DATASETS/PFE/donnees_entrainement_k_24h_history_v2


Traitement des arcs:   0%|          | 0/1539 [00:00<?, ?it/s]


Traitement terminé.
1493 fichiers ont été créés dans drive/MyDrive/DATASETS/PFE/donnees_entrainement_k_24h_history_v2.
